# Analyze the TIME / ESS distribution 

Randomly choose 25 samples and evaluate the model's Time / ESS 

### Import the libraries 

In [10]:
# Standard library imports
import sys

import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['KERAS_BACKEND'] = 'tensorflow'

# Third-party library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import arviz as az
import timeit

import scipy.stats as stats
from keras.models import Model as Model_nn
from keras.models import Sequential, load_model
from keras.layers import Dense, Concatenate
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Input, Dense, Add

#Try with TinyDA
import tinyDA as tda
from scipy.stats import multivariate_normal
from scipy.stats import uniform
from itertools import product


# Local module imports
sys.path.append('../../')
sys.path.append('../../solver')
#sys.path.append('./src/InverseProblems')
#sys.path.append('./src/utils')
from utils import * 
from plotting import *
from random_process import *
from model import *

### Choose the 25 random samples

In [11]:
n = 25
np.random.seed(2109)
random_samples = np.random.randint(0, 160, n)
random_samples

array([57, 32, 55, 69,  3])

### Load the data and surrogate model 

In [12]:
# Extract test data for visualization or further processing
n_eig = 64
X_values = np.loadtxt('../../data/50-25-10/X_test_50resolution.csv', delimiter = ',')
y_values = np.loadtxt('../../data/50-25-10/y_test_50resolution.csv',delimiter = ',')

In [13]:
# Choose the model parameters 
n_samples_lf = 64000
coeff_lf = 1e-09

# Initialize the neural network model
model_lf = Sequential([
    Dense(256, input_shape=(X_values.shape[1],), activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(25, activation='linear')
])

model_lf = load_model(f'../models/model_25resolution_{n_samples_lf}samples_1.keras')

# Choose the model parameters 
n_samples = 16000
coeff = 1e-08

n_neurons = 256
# Initialize the neural network model
# Define the three branches of the model
input_params = Input(shape=(X_values.shape[1],))
input_pod = Input(shape=(y_values.shape[1],))

# Define the first branch (parameters)
x1 = Dense(n_neurons, activation='gelu')(input_params)
x1 = Dense(n_neurons, activation='gelu')(x1)
x1 = Dense(n_neurons, activation='gelu')(x1)
x1 = Dense(n_neurons, activation='gelu')(x1)
x1 = Dense(n_neurons, activation='gelu')(x1)
x1 = Dense(n_neurons, activation='gelu')(x1)

# Define the second branch (POD)
x2 = Dense(n_neurons, activation='gelu')(input_pod)

# # Define the second branch (POD)
# x3 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(w))(input_nn)

# Combine the outputs of the three branches
combined = Add()([x1,x2])
combined = Dense(n_neurons, activation='gelu')(combined)
output = Dense(25, activation='linear')(combined)

# Create the model
model_hf = Model_nn(inputs=[input_params,input_pod], outputs=output)
model_hf = load_model(f'..//models/model_2step_50-25resolution_{n_samples}samples_1.keras')

model_nn = lambda input: np.array(model_hf([input.reshape(1,64), model_lf(input.reshape(1,64)).numpy()])).reshape(25)

/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Time / ESS noise 0.001 and multiplicative coefficient 

In [14]:
noise = 0.001
scaling = 0.015 #0.05
n_iter = 55000
burnin = 5000
thin = 50

Times = []
Time_ESS = []
ESS = []
i = 1

# Define the prior distribution and the proposal (common to all samples)
x_distribution = stats.multivariate_normal(mean = np.zeros(64), cov = np.eye(64))
my_proposal = tda.CrankNicolson(scaling=scaling, adaptive=False, gamma = 1.01, period=100)

for sample in random_samples:
    print('Sample = ', sample)
    x_true = X_values[sample]
    y_true = y_values[sample]

    y_observed = y_true + np.random.normal(scale=noise,size=y_true.shape[0])

    # LIKELYHOOD
    cov_likelihood = noise**2 * np.eye(25)
    y_distribution = tda.GaussianLogLike(y_observed, cov_likelihood*10)  

    # initialise the Posterior
    my_posterior = tda.Posterior(x_distribution, y_distribution, model_nn)

    # RUN THE MCMC
    start = timeit.default_timer()
    samples = tda.sample(my_posterior, my_proposal, iterations=n_iter, n_chains=1, initial_parameters=np.zeros(64))
    end = timeit.default_timer()

    # Remove the burnin and sub-sample
    idata = tda.to_inference_data(samples, level='fine')
    idata = idata.sel(draw=slice(burnin, None, thin), groups="posterior")
    ess = az.ess(idata)

    #Compute the time
    t = end-start
    Times.append(t)

    # Compute the mean ESS on the 64 parameters
    e = np.mean([ess.data_vars['x'+str(i)].values for i in range(64)])
    ESS.append(e)

    #Compute Time / ESS
    Time_ESS.append(t/e)
    
    print('Time:', t, '   ESS: ', e, '   Time/ESS: ',t/e )

# Save the results 
# Specify the folder path (assuming it already exists)
folder_path = './recorded_values'  # Replace with your actual path

# Save the file in the specified folder
file_path = os.path.join(folder_path, 'MF_2step_time_ess_001.npy')
np.save(file_path, Time_ESS)
file_path = os.path.join(folder_path, 'MF_2step_Times_001.npy')
np.save(file_path, Times)
file_path = os.path.join(folder_path, 'MF_2step_ESS_001.npy')
np.save(file_path, ESS)

Sample =  57
Sampling chain 1/1


Running chain, α = 0.38: 100%|██████████| 1000/1000 [00:12<00:00, 78.10it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 12.829833374998998    ESS:  4.028678647507794    Time/ESS:  3.1846256546016
Sample =  32
Sampling chain 1/1


Running chain, α = 0.42: 100%|██████████| 1000/1000 [00:11<00:00, 86.20it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 11.614287625008728    ESS:  4.77029687976076    Time/ESS:  2.434709603564801
Sample =  55
Sampling chain 1/1


...

Time: 12.315154000010807    ESS:  4.278714736695965    Time/ESS:  2.878236750487508
Sample =  69
Sampling chain 1/1


Running chain, α = 0.36: 100%|██████████| 1000/1000 [00:12<00:00, 78.55it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 12.746541125001386    ESS:  4.2917839669727105    Time/ESS:  2.9699866589492845
Sample =  3
Sampling chain 1/1


Running chain, α = 0.42: 100%|██████████| 1000/1000 [00:14<00:00, 71.00it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 14.103937458974542    ESS:  4.997559777259492    Time/ESS:  2.8221648339559646
Effective Sample Size:  <xarray.Dataset> Size: 512B
Dimensions:  ()
Data variables: (12/64)
    x0       float64 8B 5.681
    x1       float64 8B 5.14
    x2       float64 8B 2.094
    x3       float64 8B 2.071
    x4       float64 8B 2.1
    x5       float64 8B 2.162
    ...       ...
    x58      float64 8B 3.579
    x59      float64 8B 13.77
    x60      float64 8B 10.27
    x61      float64 8B 5.43
    x62      float64 8B 2.489
    x63      float64 8B 2.272 





### Time/ESS Higher Noise

In [16]:
noise = 0.01
scaling = 0.04
n_iter = 55000
burnin = 5000
thin = 50

Times = []
Time_ESS = []
ESS = []
i = 1

# Define the prior distribution and the proposal (common to all samples)
x_distribution = stats.multivariate_normal(mean = np.zeros(64), cov = np.eye(64))
my_proposal = tda.CrankNicolson(scaling=scaling, adaptive=False, gamma = 1.01, period=100)

for sample in random_samples:
    print('Sample = ', sample)
    x_true = X_values[sample]
    y_true = y_values[sample]

    y_observed = y_true + np.random.normal(scale=noise,size=y_true.shape[0])

    # LIKELYHOOD
    cov_likelihood = noise**2 * np.eye(25)
    y_distribution = tda.GaussianLogLike(y_observed, cov_likelihood)  

    # initialise the Posterior
    my_posterior = tda.Posterior(x_distribution, y_distribution, model_nn)

    # RUN THE MCMC
    start = timeit.default_timer()
    samples = tda.sample(my_posterior, my_proposal, iterations=n_iter, n_chains=1, initial_parameters=np.zeros(64))
    end = timeit.default_timer()

    # Remove the burnin and sub-sample
    idata = tda.to_inference_data(samples, level='fine')
    idata = idata.sel(draw=slice(burnin, None, thin), groups="posterior")
    ess = az.ess(idata)

    #Compute the time
    t = end-start
    Times.append(t)

    # Compute the mean ESS on the 64 parameters
    e = np.mean([ess.data_vars['x'+str(i)].values for i in range(64)])
    ESS.append(e)

    #Compute Time / ESS
    Time_ESS.append(t/e)
    
    print('Time:', t, '   ESS: ', e, '   Time/ESS: ',t/e , '     ', i,'/', len(random_samples))

    i = i+1

# Save the results 
# Specify the folder path (assuming it already exists)
folder_path = './recorded_values'  # Replace with your actual path

# Save the file in the specified folder
file_path = os.path.join(folder_path, 'MF_2step_time_ess_01.npy')
np.save(file_path, Time_ESS)
file_path = os.path.join(folder_path, 'MF_2step_Times_01.npy')
np.save(file_path, Times)
file_path = os.path.join(folder_path, 'MF_2step_ESS_01.npy')
np.save(file_path, ESS)

Sample =  57
Sampling chain 1/1


Running chain, α = 0.33: 100%|██████████| 1000/1000 [00:15<00:00, 63.94it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 15.717119583016029    ESS:  5.351230111037505    Time/ESS:  2.9371040409190643       1 / 5
Sample =  32
Sampling chain 1/1


Running chain, α = 0.33: 100%|██████████| 1000/1000 [00:14<00:00, 67.00it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 14.94473712501349    ESS:  5.130320605171551    Time/ESS:  2.913022065316668       2 / 5
Sample =  55
Sampling chain 1/1


...

Time: 13.679101499990793    ESS:  4.399217551777843    Time/ESS:  3.1094396535271818       3 / 5
Sample =  69
Sampling chain 1/1


Running chain, α = 0.35: 100%|██████████| 1000/1000 [00:12<00:00, 77.97it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 12.840962916990975    ESS:  5.400446233897606    Time/ESS:  2.377759607417368       4 / 5
Sample =  3
Sampling chain 1/1


Running chain, α = 0.38: 100%|██████████| 1000/1000 [00:12<00:00, 77.38it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 12.939926541002933    ESS:  5.842866202901444    Time/ESS:  2.2146539201218127       5 / 5
Effective Sample Size:  <xarray.Dataset> Size: 512B
Dimensions:  ()
Data variables: (12/64)
    x0       float64 8B 8.679
    x1       float64 8B 2.373
    x2       float64 8B 10.47
    x3       float64 8B 2.562
    x4       float64 8B 4.334
    x5       float64 8B 5.95
    ...       ...
    x58      float64 8B 11.17
    x59      float64 8B 2.899
    x60      float64 8B 11.93
    x61      float64 8B 2.114
    x62      float64 8B 2.109
    x63      float64 8B 2.618 



